In [ ]:
import rasterio
import numpy as np
import cv2
from rasterio.features import shapes
from shapely.geometry import shape, mapping
from scipy import ndimage
import matplotlib.pyplot as plt
from pathlib import Path
import json

# --- Configuration ---
mask_path = "outputs/tile_20480_40960_masks.tif"
clean_mask_path = "masks_clean1.tif"
boundaries_path = "masks_boundaries1.tif"
polygons_geojson = "polygons.geojson1"

# --- Watershed Separation (for touching objects) ---
USE_WATERSHED_SEPARATION = True  # Separate touching objects using watershed
WATERSHED_THRESHOLD = 0.2        # Lower = more aggressive separation (0.1-0.5)

# --- Morphological Processing ---
USE_MORPHOLOGY = True        # Apply morphological operations to improve shapes
MORPH_CLOSE_KERNEL = 2       # Kernel size for closing (fill small holes)
MORPH_CLOSE_ITERATIONS = 1   # How many times to apply closing
MORPH_DILATE_KERNEL = 3      # Kernel size for dilation (expand objects)
MORPH_DILATE_ITERATIONS = 1  # How many times to dilate
MORPH_ERODE_KERNEL = 3       # Kernel size for erosion (shrink back slightly)
MORPH_ERODE_ITERATIONS = 1   # How many times to erode (use to smooth after dilation)

# --- Filtering Parameters ---
MIN_AREA = 2000              # Minimum area in pixels (increase to remove smaller blobs)
MAX_AREA = 5000000          # Maximum area (remove if too large)

# Shape quality filters
MIN_COMPACTNESS = 0.01       # Remove irregular shapes (0-1, higher = more compact)
MIN_SOLIDITY = 0.2          # Remove fragmented shapes (0-1, area/convex_hull)
MAX_ASPECT_RATIO = 200.0      # Remove elongated shapes (width/height ratio)
MIN_CONVEXITY = 0.3         # Remove concave/irregular shapes (0-1)
MIN_EXTENT = 0.05            # Remove thin/sparse shapes (area/bounding_box)

BOUNDARY_THICKNESS = 4      
CREATE_BOUNDARY_FILE = True
SAVE_GEOJSON = True

def load_sam_mask(mask_path):
    """Load SAM mask and convert to instance mask"""
    with rasterio.open(mask_path) as src:
        mask_data = src.read()
        profile = src.profile.copy()
        transform = src.transform
        crs = src.crs
        
    print(f"\n{'='*60}")
    print(f"MASK ANALYSIS")
    print(f"{'='*60}")
    print(f"Shape: {mask_data.shape}")
    print(f"Dtype: {mask_data.dtype}")
    print(f"Bands: {mask_data.shape[0] if mask_data.ndim == 3 else 1}")
    
    # Handle multi-band masks
    if mask_data.ndim == 3 and mask_data.shape[0] > 1:
        print(f"\n✓ Multi-band mask: {mask_data.shape[0]} bands")
        
        instance_mask = np.zeros(mask_data.shape[1:], dtype=np.uint16)
        
        print(f"\nAnalyzing bands:")
        valid_bands = 0
        for band_idx in range(mask_data.shape[0]):
            band = mask_data[band_idx]
            nonzero = np.count_nonzero(band > 0)
            
            if nonzero > 0:
                valid_bands += 1
                band_mask = band > 0
                instance_mask[band_mask & (instance_mask == 0)] = band_idx + 1
                
                if band_idx < 10:
                    print(f"  Band {band_idx+1}: {nonzero:,} pixels")
        
        if mask_data.shape[0] > 10:
            print(f"  ... and {mask_data.shape[0] - 10} more bands")
        
        print(f"\n✓ Converted {valid_bands} bands")
        
    else:
        instance_mask = mask_data.squeeze()
        unique_vals = np.unique(instance_mask)
        unique_vals = unique_vals[unique_vals > 0]
        
        if len(unique_vals) > 1:
            print(f"\n✓ Single-band instance mask: {len(unique_vals)} objects")
        else:
            print(f"\n⚠ Binary mask - separating connected regions")
            instance_mask, num = ndimage.label(instance_mask > 0)
            print(f"→ Found {num} connected regions")
    
    num_instances = len(np.unique(instance_mask)) - 1
    total_pixels = np.sum(instance_mask > 0)
    
    print(f"\n{'='*60}")
    print(f"RESULT: {num_instances} objects, {total_pixels:,} pixels")
    print(f"{'='*60}")
    
    return instance_mask, profile, transform, crs

def apply_watershed_separation(instance_mask):
    """
    Apply watershed algorithm to separate touching objects
    This is useful when SAM detects objects that are touching/overlapping
    
    Parameters:
    -----------
    instance_mask : numpy.ndarray
        Instance mask where objects might be touching
    
    Returns:
    --------
    numpy.ndarray : Instance mask with separated objects
    """
    if not USE_WATERSHED_SEPARATION:
        return instance_mask
    
    print(f"\n{'='*60}")
    print(f"WATERSHED SEPARATION (Separate Touching Objects)")
    print(f"{'='*60}")
    
    # Create binary mask of all objects
    binary_mask = (instance_mask > 0).astype(np.uint8)
    
    # Distance transform to find object centers
    dist = cv2.distanceTransform(binary_mask, cv2.DIST_L2, 3)
    
    # Normalize distance
    dist_norm = cv2.normalize(dist, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    # Find local maxima (object centers) with configurable threshold
    _, peaks = cv2.threshold(dist_norm, WATERSHED_THRESHOLD * dist_norm.max(), 255, cv2.THRESH_BINARY)
    peaks = peaks.astype(np.uint8)
    
    # Label the peaks (these become watershed markers)
    _, markers = cv2.connectedComponents(peaks)
    
    # Small dilation of markers to ensure they're inside objects
    kernel = np.ones((2, 2), np.uint8)
    markers = cv2.dilate(markers.astype(np.uint8), kernel, iterations=1)
    markers = markers.astype(np.int32)
    
    # Mark background
    markers[binary_mask == 0] = 0
    
    # Apply watershed
    binary_3ch = cv2.cvtColor(binary_mask * 255, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(binary_3ch, markers)
    
    # Create separated instance mask
    # Watershed boundaries are marked as -1, we set them to 0
    separated_mask = np.where(markers > 0, markers, 0).astype(np.uint16)
    
    # Count objects before and after
    before_count = len(np.unique(instance_mask)) - 1
    after_count = len(np.unique(separated_mask)) - 1
    
    print(f"✓ Watershed separation applied:")
    print(f"  Threshold: {WATERSHED_THRESHOLD}")
    print(f"  Objects before: {before_count}")
    print(f"  Objects after: {after_count}")
    print(f"  New objects separated: {after_count - before_count}")
    
    return separated_mask

def apply_morphological_operations(instance_mask):
    """
    Apply morphological operations to improve object shapes
    - Closing: fills small holes and gaps
    - Dilation: expands objects, smooths edges
    - Erosion: shrinks objects back (optional, for smoothing)
    """
    if not USE_MORPHOLOGY:
        return instance_mask
    
    print(f"\n{'='*60}")
    print(f"MORPHOLOGICAL PROCESSING")
    print(f"{'='*60}")
    
    unique_ids = np.unique(instance_mask)
    unique_ids = unique_ids[unique_ids > 0]
    
    improved_mask = np.zeros_like(instance_mask, dtype=np.uint16)
    
    for inst_id in unique_ids:
        obj_mask = (instance_mask == inst_id).astype(np.uint8)
        
        # 1. Morphological Closing (fill small holes and gaps)
        if MORPH_CLOSE_KERNEL > 0 and MORPH_CLOSE_ITERATIONS > 0:
            close_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, 
                                                     (MORPH_CLOSE_KERNEL, MORPH_CLOSE_KERNEL))
            obj_mask = cv2.morphologyEx(obj_mask, cv2.MORPH_CLOSE, close_kernel, 
                                       iterations=MORPH_CLOSE_ITERATIONS)
        
        # 2. Dilation (expand object, smooth edges)
        if MORPH_DILATE_KERNEL > 0 and MORPH_DILATE_ITERATIONS > 0:
            dilate_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, 
                                                      (MORPH_DILATE_KERNEL, MORPH_DILATE_KERNEL))
            obj_mask = cv2.dilate(obj_mask, dilate_kernel, iterations=MORPH_DILATE_ITERATIONS)
        
        # 3. Erosion (shrink back slightly to smooth, optional)
        if MORPH_ERODE_KERNEL > 0 and MORPH_ERODE_ITERATIONS > 0:
            erode_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, 
                                                     (MORPH_ERODE_KERNEL, MORPH_ERODE_KERNEL))
            obj_mask = cv2.erode(obj_mask, erode_kernel, iterations=MORPH_ERODE_ITERATIONS)
        
        # Assign to output mask
        improved_mask[obj_mask > 0] = inst_id
    
    print(f"✓ Applied morphological operations:")
    if MORPH_CLOSE_KERNEL > 0:
        print(f"  • Closing: kernel={MORPH_CLOSE_KERNEL}, iterations={MORPH_CLOSE_ITERATIONS}")
    if MORPH_DILATE_KERNEL > 0:
        print(f"  • Dilation: kernel={MORPH_DILATE_KERNEL}, iterations={MORPH_DILATE_ITERATIONS}")
    if MORPH_ERODE_KERNEL > 0:
        print(f"  • Erosion: kernel={MORPH_ERODE_KERNEL}, iterations={MORPH_ERODE_ITERATIONS}")
    
    # Show before/after stats
    before_pixels = np.sum(instance_mask > 0)
    after_pixels = np.sum(improved_mask > 0)
    change = after_pixels - before_pixels
    change_pct = (change / before_pixels * 100) if before_pixels > 0 else 0
    
    print(f"\n📊 Pixel changes:")
    print(f"  Before: {before_pixels:,} pixels")
    print(f"  After: {after_pixels:,} pixels")
    print(f"  Change: {change:+,} pixels ({change_pct:+.1f}%)")
    
    return improved_mask

def calculate_shape_metrics(obj_mask):
    """
    Calculate shape quality metrics for filtering
    
    Returns:
    --------
    dict : Shape metrics (area, compactness, solidity, aspect_ratio, etc.)
    """
    # Find contours
    contours, _ = cv2.findContours(obj_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    contour = max(contours, key=cv2.contourArea)
    
    # Basic metrics
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    if area == 0 or perimeter == 0:
        return None
    
    # Bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    
    # Convex hull
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    
    # Calculate metrics
    metrics = {
        'area': area,
        'perimeter': perimeter,
        'width': w,
        'height': h,
    }
    
    # Compactness (circularity): 4π*area/perimeter²
    # Perfect circle = 1.0, irregular = closer to 0
    metrics['compactness'] = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0
    
    # Solidity: area / convex_hull_area
    # Measures how "solid" the shape is (no concavities)
    metrics['solidity'] = area / hull_area if hull_area > 0 else 0
    
    # Aspect ratio: width / height
    metrics['aspect_ratio'] = max(w, h) / min(w, h) if min(w, h) > 0 else 0
    
    # Extent: area / bounding_box_area
    # Measures how much of the bounding box is filled
    bbox_area = w * h
    metrics['extent'] = area / bbox_area if bbox_area > 0 else 0
    
    # Convexity: convex_hull_perimeter / perimeter
    metrics['convexity'] = hull_perimeter / perimeter if perimeter > 0 else 0
    
    return metrics

def filter_objects_by_quality(instance_mask):
    """
    Filter objects based on size and shape quality
    """
    unique_ids = np.unique(instance_mask)
    unique_ids = unique_ids[unique_ids > 0]
    
    cleaned_mask = np.zeros_like(instance_mask, dtype=np.uint16)
    new_id = 1
    
    # Statistics
    removed_reasons = {
        'too_small': 0,
        'too_large': 0,
        'low_compactness': 0,
        'low_solidity': 0,
        'high_aspect_ratio': 0,
        'low_convexity': 0,
        'low_extent': 0
    }
    
    print(f"\n{'='*60}")
    print(f"FILTERING OBJECTS")
    print(f"{'='*60}")
    print(f"Criteria:")
    print(f"  Area: {MIN_AREA:,} - {MAX_AREA:,} pixels")
    print(f"  Compactness: ≥ {MIN_COMPACTNESS:.2f}")
    print(f"  Solidity: ≥ {MIN_SOLIDITY:.2f}")
    print(f"  Aspect ratio: ≤ {MAX_ASPECT_RATIO:.1f}")
    print(f"  Convexity: ≥ {MIN_CONVEXITY:.2f}")
    print(f"  Extent: ≥ {MIN_EXTENT:.2f}")
    
    kept_objects = []
    
    for inst_id in unique_ids:
        obj_mask = (instance_mask == inst_id).astype(np.uint8)
        
        # Calculate shape metrics
        metrics = calculate_shape_metrics(obj_mask)
        
        if metrics is None:
            removed_reasons['too_small'] += 1
            continue
        
        # Apply filters
        keep = True
        reason = None
        
        # Area filter
        if metrics['area'] < MIN_AREA:
            keep = False
            reason = 'too_small'
        elif metrics['area'] > MAX_AREA:
            keep = False
            reason = 'too_large'
        # Compactness filter (remove irregular blobs)
        elif metrics['compactness'] < MIN_COMPACTNESS:
            keep = False
            reason = 'low_compactness'
        # Solidity filter (remove fragmented shapes)
        elif metrics['solidity'] < MIN_SOLIDITY:
            keep = False
            reason = 'low_solidity'
        # Aspect ratio filter (remove elongated shapes)
        elif metrics['aspect_ratio'] > MAX_ASPECT_RATIO:
            keep = False
            reason = 'high_aspect_ratio'
        # Convexity filter (remove very concave shapes)
        elif metrics['convexity'] < MIN_CONVEXITY:
            keep = False
            reason = 'low_convexity'
        # Extent filter (remove sparse/thin shapes)
        elif metrics['extent'] < MIN_EXTENT:
            keep = False
            reason = 'low_extent'
        
        if keep:
            cleaned_mask[obj_mask > 0] = new_id
            kept_objects.append(metrics)
            new_id += 1
        else:
            if reason:
                removed_reasons[reason] += 1
    
    total_removed = sum(removed_reasons.values())
    total_kept = new_id - 1
    
    print(f"\n📊 Results:")
    print(f"  ✓ Kept: {total_kept} objects")
    print(f"  ✗ Removed: {total_removed} objects")
    
    if total_removed > 0:
        print(f"\n📋 Removal reasons:")
        for reason, count in removed_reasons.items():
            if count > 0:
                print(f"  • {reason.replace('_', ' ').title()}: {count}")
    
    if kept_objects:
        areas = [m['area'] for m in kept_objects]
        compactness = [m['compactness'] for m in kept_objects]
        
        print(f"\n📈 Kept objects stats:")
        print(f"  Area: {min(areas):,.0f} - {max(areas):,.0f} px (avg: {np.mean(areas):,.0f})")
        print(f"  Compactness: {min(compactness):.3f} - {max(compactness):.3f} (avg: {np.mean(compactness):.3f})")
    
    return cleaned_mask

def extract_polygons_per_object(instance_mask, transform):
    """Extract ONE polygon per object"""
    unique_ids = np.unique(instance_mask)
    unique_ids = unique_ids[unique_ids > 0]
    
    polygons = []
    
    print(f"\n{'='*60}")
    print(f"EXTRACTING POLYGONS")
    print(f"{'='*60}")
    
    for inst_id in unique_ids:
        obj_mask = (instance_mask == inst_id).astype(np.uint8)
        area = np.sum(obj_mask)
        
        # Extract polygon
        for geom, val in shapes(obj_mask, mask=obj_mask, transform=transform):
            if val > 0:
                poly = shape(geom)
                polygons.append({
                    'polygon': poly,
                    'id': int(inst_id),
                    'area': int(area)
                })
                break
    
    print(f"✓ Extracted {len(polygons)} polygons")
    
    if polygons:
        areas = [p['area'] for p in polygons]
        print(f"\nPolygon Statistics:")
        print(f"  Count: {len(polygons)}")
        print(f"  Area range: {min(areas):,} - {max(areas):,} pixels")
        print(f"  Average: {np.mean(areas):,.0f} pixels")
    
    return polygons

def save_geojson(polygons, output_path, crs):
    """Save polygons to GeoJSON"""
    features = []
    
    for poly_data in polygons:
        feature = {
            "type": "Feature",
            "properties": {
                "id": poly_data['id'],
                "area_pixels": poly_data['area']
            },
            "geometry": mapping(poly_data['polygon'])
        }
        features.append(feature)
    
    geojson = {
        "type": "FeatureCollection",
        "crs": {
            "type": "name",
            "properties": {"name": str(crs) if crs else "EPSG:4326"}
        },
        "features": features
    }
    
    with open(output_path, 'w') as f:
        json.dump(geojson, f, indent=2)
    
    print(f"\n✓ GeoJSON saved: {output_path}")
    print(f"  → {len(features)} polygons")

def extract_boundaries(instance_mask, thickness=4):
    """Extract boundary for each object"""
    unique_ids = np.unique(instance_mask)
    unique_ids = unique_ids[unique_ids > 0]
    
    boundaries = np.zeros_like(instance_mask, dtype=np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    
    for inst_id in unique_ids:
        obj_mask = (instance_mask == inst_id).astype(np.uint8)
        edges = cv2.morphologyEx(obj_mask, cv2.MORPH_GRADIENT, kernel)
        boundaries = np.maximum(boundaries, edges)
    
    if thickness > 1:
        thick_kernel = np.ones((thickness, thickness), np.uint8)
        boundaries = cv2.dilate(boundaries, thick_kernel, iterations=1)
    
    return boundaries

def visualize_results(instance_mask, boundaries, polygons, save_path):
    """Create visualization"""
    binary_mask = (instance_mask > 0).astype(np.uint8)
    num_objects = len(np.unique(instance_mask)) - 1
    
    fig, axs = plt.subplots(2, 2, figsize=(18, 18))
    
    # 1. Instance mask (color-coded)
    axs[0, 0].imshow(instance_mask, cmap="nipy_spectral", interpolation='nearest')
    processing_text = []
    if USE_WATERSHED_SEPARATION:
        processing_text.append("Watershed")
    if USE_MORPHOLOGY:
        processing_text.append("Morphology")
    title_suffix = f" ({' + '.join(processing_text)})" if processing_text else ""
    axs[0, 0].set_title(f"Instance Mask{title_suffix}\n({num_objects} high-quality objects)", 
                        fontsize=14, fontweight='bold')
    axs[0, 0].axis("off")
    
    # 2. With boundaries overlay
    rgb = np.stack([binary_mask, binary_mask, binary_mask], axis=-1).astype(float)
    rgb[boundaries > 0] = [1.0, 0, 0]
    
    axs[0, 1].imshow(rgb, interpolation='nearest')
    axs[0, 1].set_title(f"Objects with Boundaries\n({len(polygons)} polygons, boundaries in RED)", 
                        fontsize=14, fontweight='bold', color='darkgreen')
    axs[0, 1].axis("off")
    
    # 3. Binary mask
    axs[1, 0].imshow(binary_mask, cmap="gray", interpolation='nearest')
    axs[1, 0].set_title(f"Binary Mask\n(All filtered objects)", 
                        fontsize=14, fontweight='bold')
    axs[1, 0].axis("off")
    
    # 4. Boundaries only
    boundary_display = np.zeros_like(instance_mask, dtype=np.uint8)
    boundary_display[boundaries > 0] = 255
    
    axs[1, 1].imshow(boundary_display, cmap="gray", interpolation='nearest')
    axs[1, 1].set_title(f"Boundaries Only\n({BOUNDARY_THICKNESS}px thickness)", 
                        fontsize=14, fontweight='bold')
    axs[1, 1].axis("off")
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Visualization saved: {save_path}")
    plt.show()

def main():
    """Main pipeline"""
    
    if not Path(mask_path).exists():
        raise FileNotFoundError(f"Mask file not found: {mask_path}")
    
    print(f"\n{'='*60}")
    print(f"SAM MASK → FILTERED POLYGONS")
    print(f"{'='*60}")
    print(f"\nInput: {mask_path}")
    
    # Load SAM mask
    instance_mask, profile, transform, crs = load_sam_mask(mask_path)
    
    # Apply watershed separation to separate touching objects
    instance_mask = apply_watershed_separation(instance_mask)
    
    # Apply morphological operations to improve shapes
    instance_mask = apply_morphological_operations(instance_mask)
    
    # Filter objects by size and shape quality
    instance_mask = filter_objects_by_quality(instance_mask)
    
    # Extract polygons (one per object)
    polygons = extract_polygons_per_object(instance_mask, transform)
    
    # Extract boundaries
    print(f"\n{'='*60}")
    print(f"EXTRACTING BOUNDARIES")
    print(f"{'='*60}")
    print(f"Thickness: {BOUNDARY_THICKNESS}px")
    boundaries = extract_boundaries(instance_mask, BOUNDARY_THICKNESS)
    print(f"✓ Boundary pixels: {np.sum(boundaries > 0):,}")
    
    # Save files
    profile.update({
        'count': 1,
        'dtype': 'uint16',
        'compress': 'lzw',
        'nodata': 0
    })
    
    print(f"\n{'='*60}")
    print(f"SAVING FILES")
    print(f"{'='*60}")
    
    with rasterio.open(clean_mask_path, "w", **profile) as dst:
        dst.write(instance_mask.astype(np.uint16), 1)
    print(f"✓ Instance mask: {clean_mask_path}")
    
    if CREATE_BOUNDARY_FILE:
        profile_boundary = profile.copy()
        profile_boundary['dtype'] = 'uint8'
        
        with rasterio.open(boundaries_path, "w", **profile_boundary) as dst:
            dst.write(boundaries, 1)
        print(f"✓ Boundaries: {boundaries_path}")
    
    if SAVE_GEOJSON:
        save_geojson(polygons, polygons_geojson, crs)
    
    # Visualize
    print(f"\n{'='*60}")
    print(f"GENERATING VISUALIZATION")
    print(f"{'='*60}")
    visualize_results(instance_mask, boundaries, polygons, "polygons_visualization.png")
    
    print(f"\n{'='*60}")
    print(f"✓ COMPLETE!")
    print(f"{'='*60}")
    print(f"\nOutput files:")
    print(f"  1. {clean_mask_path} - Filtered instance mask")
    print(f"  2. {boundaries_path} - Boundaries")
    print(f"  3. {polygons_geojson} - Polygons (GeoJSON) ← USE THIS!")
    print(f"  4. polygons_visualization.png - Visualization")
    
    print(f"\n✓ Processing applied:")
    if USE_WATERSHED_SEPARATION:
        print(f"  • Watershed separation: separated touching objects")
    if USE_MORPHOLOGY:
        print(f"  • Morphological processing: filled holes and smoothed edges")
    print(f"  • Quality filtering: removed small/irregular shapes")
    print(f"\n✓ Only high-quality objects with proper shapes!")
    
    return instance_mask, boundaries, polygons

if __name__ == "__main__":
    try:
        instance_mask, boundaries, polygons = main()
    except Exception as e:
        print(f"\nERROR: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# --- Configuration ---
polygons_geojson = "polygons.geojson1"
original_image_path = "tiff_testing/tile_20480_40960.tif"

GRID_COLS = 4  # Number of columns in the grid
POLYGON_COLOR = 'red'  # Boundary color
LINE_WIDTH = 2  # Boundary thickness
SHOW_POLYGON_ID = True  # Show polygon ID in title

def load_polygons(geojson_path):
    """Load polygons from GeoJSON file"""
    print(f"Loading polygons from: {geojson_path}")
    gdf = gpd.read_file(geojson_path)
    print(f"✓ Loaded {len(gdf)} polygons")
    
    # Show some info
    if 'area_pixels' in gdf.columns:
        print(f"\nPolygon Statistics:")
        print(f"  Area range: {gdf['area_pixels'].min():,} - {gdf['area_pixels'].max():,} pixels")
        print(f"  Average area: {gdf['area_pixels'].mean():,.0f} pixels")
    
    return gdf

def load_original_image(image_path):
    """Load original raster image"""
    print(f"\nLoading original image: {image_path}")
    
    with rasterio.open(image_path) as src:
        img_data = src.read()
        img_transform = src.transform
        img_crs = src.crs
        
        # Calculate extent for plotting
        img_extent = [
            img_transform[2],  # left
            img_transform[2] + img_transform[0] * src.width,  # right
            img_transform[5] + img_transform[4] * src.height,  # bottom
            img_transform[5]  # top
        ]
    
    print(f"✓ Image shape: {img_data.shape}")
    print(f"✓ Image CRS: {img_crs}")
    
    # Prepare RGB or grayscale display
    if img_data.shape[0] >= 3:
        # RGB image
        rgb = np.transpose(img_data[:3], (1, 2, 0))
        
        # Normalize to 0-255 if needed
        if rgb.max() > 255:
            rgb = (rgb / rgb.max() * 255).astype(np.uint8)
        
        display_img = rgb
        cmap = None
    else:
        # Grayscale
        display_img = img_data[0]
        cmap = 'gray'
    
    return display_img, img_extent, cmap

def visualize_polygons_grid(gdf, display_img, img_extent, cmap=None, 
                           n_cols=4, figsize_per_plot=(4, 4)):
    """
    Visualize each polygon separately in a grid
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        Polygons to visualize
    display_img : numpy.ndarray
        Image to display as background
    img_extent : list
        [left, right, bottom, top] extent for image
    cmap : str, optional
        Colormap for grayscale images
    n_cols : int
        Number of columns in grid
    figsize_per_plot : tuple
        Size of each subplot
    """
    n_polygons = len(gdf)
    n_rows = int(np.ceil(n_polygons / n_cols))
    
    fig_width = figsize_per_plot[0] * n_cols
    fig_height = figsize_per_plot[1] * n_rows
    
    print(f"\nCreating visualization grid:")
    print(f"  Grid: {n_rows} rows × {n_cols} cols")
    print(f"  Figure size: {fig_width} × {fig_height}")
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height))
    axes = axes.flatten() if n_polygons > 1 else [axes]
    
    # Plot each polygon
    for idx, (_, row) in enumerate(gdf.iterrows()):
        ax = axes[idx]
        
        # Display original image
        if cmap:
            ax.imshow(display_img, cmap=cmap, extent=img_extent)
        else:
            ax.imshow(display_img, extent=img_extent)
        
        # Plot this polygon
        gdf_single = gpd.GeoDataFrame(geometry=[row.geometry], crs=gdf.crs)
        gdf_single.boundary.plot(ax=ax, edgecolor=POLYGON_COLOR, linewidth=LINE_WIDTH)
        
        # Set title
        if SHOW_POLYGON_ID and 'id' in gdf.columns:
            title = f"Polygon {idx+1} (ID: {row['id']})"
        else:
            title = f"Polygon {idx+1}"
        
        # Add area to title if available
        if 'area_pixels' in gdf.columns:
            title += f"\n{row['area_pixels']:,} px"
        
        ax.set_title(title, fontsize=10, fontweight='bold')
        ax.axis("off")
    
    # Hide unused subplots
    for i in range(n_polygons, len(axes)):
        axes[i].axis("off")
    
    plt.tight_layout()
    
    # Save figure
    output_path = "polygons_grid_visualization.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Saved visualization: {output_path}")
    
    plt.show()

def visualize_polygons_overlay(gdf, display_img, img_extent, cmap=None):
    """
    Visualize all polygons overlaid on the original image
    """
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # Display original image
    if cmap:
        ax.imshow(display_img, cmap=cmap, extent=img_extent)
    else:
        ax.imshow(display_img, extent=img_extent)
    
    # Plot all polygons
    gdf.boundary.plot(ax=ax, edgecolor=POLYGON_COLOR, linewidth=LINE_WIDTH)
    
    ax.set_title(f"All {len(gdf)} Polygons Overlay", fontsize=16, fontweight='bold')
    ax.axis("off")
    
    plt.tight_layout()
    
    # Save figure
    output_path = "polygons_overlay_visualization.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Saved overlay visualization: {output_path}")
    
    plt.show()

def main():
    """Main visualization pipeline"""
    
    print("=" * 60)
    print("VISUALIZE POLYGONS FROM GEOJSON")
    print("=" * 60)
    
    # Load polygons
    gdf = load_polygons(polygons_geojson)
    
    # Load original image
    display_img, img_extent, cmap = load_original_image(original_image_path)
    
    # Visualize each polygon separately in grid
    print("\n" + "=" * 60)
    print("Creating grid visualization (one polygon per subplot)...")
    print("=" * 60)
    visualize_polygons_grid(gdf, display_img, img_extent, cmap, n_cols=GRID_COLS)
    
    # Visualize all polygons overlaid
    print("\n" + "=" * 60)
    print("Creating overlay visualization (all polygons)...")
    print("=" * 60)
    visualize_polygons_overlay(gdf, display_img, img_extent, cmap)
    
    print("\n" + "=" * 60)
    print("VISUALIZATION COMPLETE!")
    print("=" * 60)
    print("\nOutput files:")
    print("  1. polygons_grid_visualization.png - Each polygon separate")
    print("  2. polygons_overlay_visualization.png - All polygons overlaid")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"\nERROR: {e}")
        import traceback
        traceback.print_exc()